<a href="https://colab.research.google.com/github/retajalhudaifi046-ui/cheetah-vs-hyena/blob/main/cheetah_%26_hyena.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#My dataset is available in googledrive; so I am accessing my drive from colab
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
#Once this is executed, you will see your drive appearing on the left hand side

Mounted at /content/drive


In [ ]:
# Define paths to the dataset
train_dir = '/content/drive/MyDrive/cheetah vs hyaena/train'  # Updated path
test_dir = '/content/drive/MyDrive/cheetah vs hyaena/test'    # Updated path

In [ ]:
import os

# Check and print count for train/cheetah
cheetah_train_path = os.path.join(train_dir, 'cheetah')
if os.path.exists(cheetah_train_path):
    print(f"Number of cheetahs in train: {len(os.listdir(cheetah_train_path))}")
else:
    print(f"Error: Directory not found or accessible: {cheetah_train_path}")

# Check and print count for train/hyaena
hyaena_train_path = os.path.join(train_dir, 'hyaena')
if os.path.exists(hyaena_train_path):
    print(f"Number of hyaenas in train: {len(os.listdir(hyaena_train_path))}")
else:
    print(f"Error: Directory not found or accessible: {hyaena_train_path}")

# Check and print count for test/cheetah
cheetah_test_path = os.path.join(test_dir, 'cheetah')
if os.path.exists(cheetah_test_path):
    print(f"Number of cheetahs in test: {len(os.listdir(cheetah_test_path))}")
else:
    print(f"Error: Directory not found or accessible: {cheetah_test_path}")

# Check and print count for test/hyaena
hyaena_test_path = os.path.join(test_dir, 'hyaena')
if os.path.exists(hyaena_test_path):
    print(f"Number of hyaenas in test: {len(os.listdir(hyaena_test_path))}")
else:
    print(f"Error: Directory not found or accessible: {hyaena_test_path}")

Number of cheetahs in train: 659
Error: Directory not found or accessible: /content/drive/MyDrive/cheetah vs hyaena/train/hyaena
Number of cheetahs in test: 0
Error: Directory not found or accessible: /content/drive/MyDrive/cheetah vs hyaena/test/hyaena


In [ ]:
import os

# Define the base directory
base_dir = '/content/drive/MyDrive/cheetah vs hyaena'

# If your dataset follows the same Train/Test split:
print("Train Cheetahs:", len(os.listdir(os.path.join(base_dir, 'train/cheetah'))))
print("Train Hyaenas: ", len(os.listdir(os.path.join(base_dir, 'train/hyena'))))
print("Test Cheetahs: ", len(os.listdir(os.path.join(base_dir, 'test/cheetah'))))
print("Test Hyaenas:  ", len(os.listdir(os.path.join(base_dir, 'test/hyena'))))

Train Cheetahs: 659
Train Hyaenas:  900
Test Cheetahs:  0
Test Hyaenas:   0


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Create ImageDataGenerator for training set
train_datagen = ImageDataGenerator(
    rescale=1./255, #normalisation / feature scaling
    validation_split=0.2  # Split 20% of the images for validation
)

# Load and prepare training data
train_data = train_datagen.flow_from_directory(
    train_dir,
    target_size=(256,256),
    batch_size=32,
    class_mode='binary',  # 'binary' for binary classification (cats vs. dogs)
    subset='training'  # Specify 'training' for the training set
)

# Create ImageDataGenerator for validation set
validation_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2  # Note: Using the same validation split as in the training set
)

# Load and prepare validation data
validation_data = validation_datagen.flow_from_directory(
    train_dir,
    target_size=(256,256),
    batch_size=32,
    class_mode='binary',
    subset='validation'  # Specify 'validation' for the validation set
)

Found 1248 images belonging to 2 classes.
Found 311 images belonging to 2 classes.


In [ ]:
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense,Conv2D,MaxPooling2D,Flatten,BatchNormalization,UpSampling2D, Dropout

In [ ]:
# create CNN model - custom-made

model = Sequential()

model.add(Conv2D(32,kernel_size=(3,3),padding='valid',activation='relu',input_shape=(256,256,3)))  # 32 filters
model.add(MaxPooling2D(pool_size=(2,2),strides=2,padding='valid'))

model.add(Conv2D(64,kernel_size=(3,3),padding='valid',activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2),strides=2,padding='valid'))

model.add(Conv2D(128,kernel_size=(3,3),padding='valid',activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2),strides=2,padding='valid'))

model.add(Flatten())

model.add(Dense(128,activation='relu')) #feature reduction
model.add(Dense(64,activation='relu'))
model.add(Dense(1,activation='sigmoid'))  # Changed output layer to 1 unit for binary classification


/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
from keras.optimizers import Adam
model.compile(optimizer=Adam(learning_rate=0.001),loss='binary_crossentropy',metrics=['accuracy']) #binary_crossentropy - binary classification

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# 1. Setup Augmentation (helps prevent overfitting)
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    horizontal_flip=True,
    zoom_range=0.2
)

val_datagen = ImageDataGenerator(rescale=1./255)

# 2. Load data from your specific Drive folders
# Make sure these paths match your folder names exactly!
train_data = train_datagen.flow_from_directory(
    '/content/drive/MyDrive/cheetah vs hyaena/train',
    target_size=(256, 256), # Change this to match your model's input
    batch_size=32,
    class_mode='binary'    # Use 'categorical' if you have more than 2 classes
)

validation_data = val_datagen.flow_from_directory(
    '/content/drive/MyDrive/cheetah vs hyaena/test',
    target_size=(256, 256),
    batch_size=32,
    class_mode='binary'
)

Found 1559 images belonging to 2 classes.
Found 0 images belonging to 2 classes.


In [ ]:
import os

print(f"Contents of {test_dir}:")
if os.path.exists(test_dir):
    for item in os.listdir(test_dir):
        item_path = os.path.join(test_dir, item)
        if os.path.isdir(item_path):
            print(f"  [DIR] {item}")
            # Optionally, list contents of subdirectories for deeper inspection
            # for sub_item in os.listdir(item_path):
            #     print(f"    - {sub_item}")
        else:
            print(f"  [FILE] {item}")
else:
    print(f"Error: Test directory not found or accessible: {test_dir}")

Contents of /content/drive/MyDrive/cheetah vs hyaena/test:
  [DIR] cheetah
  [DIR] hyena


In [ ]:
test_datagen = ImageDataGenerator(rescale=1./255)
test_data = test_datagen.flow_from_directory(
    test_dir,
    target_size=(256,256),
    batch_size=32,
    class_mode='binary'
)

Found 0 images belonging to 2 classes.


In [35]:
from google.colab import files
uploaded = files.upload()
print(uploaded)

Saving archive (3).zip to archive (3).zip


In [ ]:
import os

# Assuming the zip file created a directory with its base name inside the destination_dir
zip_file_base_name = 'cheetah vs hyaena-20260104T083500Z-1-001' # Extract base name from uploaded variable

# Construct the new base directory where the actual train/test folders are located
updated_dataset_base_dir = os.path.join(base_dir, zip_file_base_name)

# Update train_dir and test_dir to reflect the correct paths
train_dir = os.path.join(updated_dataset_base_dir, 'train')
test_dir = os.path.join(updated_dataset_base_dir, 'test')

print(f"Updated train_dir: {train_dir}")
print(f"Updated test_dir: {test_dir}")

# Verify if the directories now exist and contain content
print(f"\nChecking updated train_dir ({train_dir}):")
if os.path.exists(train_dir):
    print(f"  Exists. Contains {len(os.listdir(train_dir))} items.")
    if len(os.listdir(train_dir)) > 0:
        print(f"  Example sub-directory: {os.listdir(train_dir)[0]}")
else:
    print("  Does NOT exist.")

print(f"Checking updated test_dir ({test_dir}):")
if os.path.exists(test_dir):
    print(f"  Exists. Contains {len(os.listdir(test_dir))} items.")
    if len(os.listdir(test_dir)) > 0:
        print(f"  Example sub-directory: {os.listdir(test_dir)[0]}")
else:
    print("  Does NOT exist.")


Updated train_dir: /content/drive/MyDrive/cheetah vs hyaena/cheetah vs hyaena-20260104T083500Z-1-001/train
Updated test_dir: /content/drive/MyDrive/cheetah vs hyaena/cheetah vs hyaena-20260104T083500Z-1-001/test

Checking updated train_dir (/content/drive/MyDrive/cheetah vs hyaena/cheetah vs hyaena-20260104T083500Z-1-001/train):
  Does NOT exist.
Checking updated test_dir (/content/drive/MyDrive/cheetah vs hyaena/cheetah vs hyaena-20260104T083500Z-1-001/test):
  Does NOT exist.


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os

# Ensure test_dir points to the correct location after extraction
# The zip file extracted 'train' and 'test' directly under base_dir
test_dir = os.path.join(base_dir, 'test')

# Re-create the test_data generator with the corrected 'test_dir'
test_datagen = ImageDataGenerator(rescale=1./255)

test_data = test_datagen.flow_from_directory(
    test_dir,
    target_size=(256,256),
    batch_size=32,
    class_mode='binary'
)

print(f"Number of samples in test_data generator: {test_data.n}")
print(f"Class labels: {test_data.class_indices}")


Found 0 images belonging to 2 classes.
Number of samples in test_data generator: 0
Class labels: {'cheetah': 0, 'hyena': 1}


In [ ]:
import zipfile
import os

# Path to the uploaded zip file
zip_file_path = '/content/cheetah vs hyaena-20260104T083500Z-1-001.zip'

# Destination directory (assuming base_dir is defined from previous cells)
# Ensure this matches where you want your 'train' and 'test' folders to be
destination_dir = base_dir # '/content/drive/MyDrive/cheetah vs hyaena'

print(f"Extracting {zip_file_path} to {destination_dir}...")

# Create the destination directory if it doesn't exist
os.makedirs(destination_dir, exist_ok=True)

with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(destination_dir)

print("Extraction complete!")


Extracting /content/cheetah vs hyaena-20260104T083500Z-1-001.zip to /content/drive/MyDrive/cheetah vs hyaena...
Extraction complete!


In [ ]:
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Re-initialize test_datagen (if not already done)
test_datagen = ImageDataGenerator(rescale=1./255)

# Re-create the test_data generator after extraction
test_data = test_datagen.flow_from_directory(
    test_dir,
    target_size=(256,256),
    batch_size=32,
    class_mode='binary'
)

print(f"Number of samples in test_data generator: {test_data.n}")
print(f"Class labels: {test_data.class_indices}")

print(f"\nInspecting contents of {test_dir}:")
if os.path.exists(test_dir):
    for item in os.listdir(test_dir):
        item_path = os.path.join(test_dir, item)
        if os.path.isdir(item_path):
            print(f"  [DIR] {item}")
            # Optionally, list contents of subdirectories for deeper inspection
            for sub_item in os.listdir(item_path):
                print(f"    - {sub_item}")
        else:
            print(f"  [FILE] {item}")
else:
    print(f"Error: Test directory not found or accessible: {test_dir}")

Found 0 images belonging to 2 classes.
Number of samples in test_data generator: 0
Class labels: {'cheetah': 0, 'hyena': 1}

Inspecting contents of /content/drive/MyDrive/cheetah vs hyaena/test:
  [DIR] cheetah
  [DIR] hyena


In [36]:
from keras.optimizers import Adam
model.compile(optimizer=Adam(learning_rate=0.001),loss='binary_crossentropy',metrics=['accuracy']) #binary_crossentropy - binary classification

In [38]:
import os

base_dir = '/content/drive/MyDrive/cheetah vs hyaena'

print(f"Train Cheetahs: {len(os.listdir(os.path.join(base_dir, 'train/cheetah')))}")
print(f"Train Hyaenas: {len(os.listdir(os.path.join(base_dir, 'train/hyena')))}")
print(f"Test Cheetahs: {len(os.listdir(os.path.join(base_dir, 'test/cheetah')))}")
print(f"Test Hyaenas: {len(os.listdir(os.path.join(base_dir, 'test/hyena')))}")

Train Cheetahs: 659
Train Hyaenas: 900
Test Cheetahs: 0
Test Hyaenas: 0


In [39]:
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense,Conv2D,MaxPooling2D,Flatten,BatchNormalization,UpSampling2D, Dropout

In [41]:
def generator_wrapper(generator):
    for x_batch, y_batch in generator:
        # Reshape y_batch from (batch_size,) to (batch_size, 1)
        yield x_batch, y_batch.reshape(-1, 1)

# Ensure both train_data and validation_data have their labels correctly shaped
history = model.fit(
    generator_wrapper(train_data),
    epochs=20,
    validation_data=generator_wrapper(validation_data)
)

Epoch 1/20


ValueError: Arguments `target` and `output` must have the same shape. Received: target.shape=(None, 1), output.shape=(None, 2)

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, BatchNormalization, Dropout
from tensorflow.keras.optimizers import Adam

# Re-create the CNN model to ensure correct configuration
model = Sequential()

model.add(Conv2D(32, kernel_size=(3,3), padding='valid', activation='relu', input_shape=(256,256,3)))
model.add(MaxPooling2D(pool_size=(2,2), strides=2, padding='valid'))

model.add(Conv2D(64, kernel_size=(3,3), padding='valid', activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2), strides=2, padding='valid'))

model.add(Conv2D(128, kernel_size=(3,3), padding='valid', activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2), strides=2, padding='valid'))

model.add(Flatten())

model.add(Dense(128, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='sigmoid')) # Ensure final layer is 1 unit for binary classification

# Re-compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

# Now, re-run the training with the updated model
def generator_wrapper(generator):
    for x_batch, y_batch in generator:
        yield x_batch, y_batch.reshape(-1, 1)

history = model.fit(
    generator_wrapper(train_data),
    epochs=20,
    validation_data=generator_wrapper(validation_data)
)


/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/20
    476/Unknown 2433s 5s/step - accuracy: 0.7362 - loss: 0.5151